<font color="#CC3D3D"><p>
# Building a DNN Model with Keras

####  전역변수 설정

In [ ]:
DNN_VERSION = 1.0  # Model version
SEED = 0           # random_state 지정

In [ ]:
#!pip install tensorflow

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random

import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(SEED)
    random.seed(SEED)
    tf.compat.v1.set_random_seed(SEED)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET")  # optional
   
reset_seeds()

#### (전처리된) 데이터 로딩

In [ ]:
# 학습 데이터
X_train = pd.read_csv('X_train.csv').drop(columns='ID')
y_train = pd.read_csv('y_train.csv').Salary

# 평가 데이터
X_test = pd.read_csv('X_test.csv')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [ ]:
X_train

In [ ]:
# 학습 데이터를 다시 학습과 검증 데이터로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=SEED)

<font color="blue"><p>
#### DNN 모형 생성

<font color="blue"><p>
- 1단계: 모형 정의하기 (입력층/은익층/출력층 쌓기)

In [ ]:
inputs = keras.Input(shape=X_train.shape[1], )          # 입력층
x = keras.layers.Dense(128, activation='relu')(inputs)  # 첫번째 은익층
x = keras.layers.Dense(64, activation='relu')(x)        # 두번째 은익층
output = keras.layers.Dense(1, activation='linear')(x)  # 출력층

model = keras.Model(inputs=inputs, outputs=output)      # 입력층과 출력층을 지정함으로써 전체모형 정의

In [ ]:
# 모형 구조 출력: Total params가 많을수록 복잡한 모델임.
model.summary()

In [ ]:
# 연결 그래프 시각화: graphviz와 pydot 설치 필요 (graphviz 다운로드하고 설치한 후 pip install pydot)
#keras.utils.plot_model(model, show_shapes=True)

<font color="blue"><p>
- 2단계: 학습방식(손실함수/옵티마이저/평가척도) 설정하기

In [ ]:
model.compile(
    loss='mse',
    optimizer=keras.optimizers.RMSprop(),                     
    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')]
)

<font color="blue"><p>
- 3단계: 모형 학습하기 (에포크, 배치사이즈, 조기중단 등 지정) 

In [ ]:
hist = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_valid,y_valid), # DNN 학습시 검증데이터가 필요함 => 미리 학습과 검증 데이터로 나누었을 경우
#    validation_split=0.1,             # DNN 학습시 검증데이터가 필요함 => 미리 나누지 않을 경우
    batch_size=64, 
    epochs=200, 
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)], # 검증데이터의 Loss가 5회연속 감소하지 않으면 학습 중단.
    shuffle=False,       # 재현을 위해 False로 설정.
    verbose=2,
)

In [ ]:
# 러닝커브 그리기: 학습이 잘되었는지 어느정도 판단할 수 있음.

plt.plot(hist.history['rmse'], label="train")
plt.plot(hist.history['val_rmse'], label="valid")
plt.xlabel('epoch')
plt.ylabel('RMSE')
plt.legend()
plt.show()

In [ ]:
results = model.evaluate(X_valid, y_valid)
print("valid loss, valid rmse:", results)

<font color="blue"><p>
- 4단계: 예측하기

In [ ]:
# 예측값 얻기: sklearn 과 달리 2차원 array로 출력 => 1차원으로 변경 필요.
pred = model.predict(X_test).flatten()

# submission 화일 생성
filename = f'dnn_{DNN_VERSION}_{hist.history["val_rmse"][-1]:.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':pred}).to_csv(filename, index=False)

print(f'{filename} is ready to submit.')

<font color="#CC3D3D"><p>
# End